In [2]:
import os
import cv2
import numpy as np
from albumentations import (
    Compose, Rotate, HorizontalFlip, VerticalFlip, 
    GaussianBlur, RandomCrop, BboxParams
)

# Set your paths
input_dir = r"C:\Users\LENOVO\Downloads\Programs\positive"
output_dir = input_dir  # same directory

# Create augmentation pipeline with more rotation and less blur
aug = Compose([
    Rotate(limit=45, p=0.8),  # High probability (80%) and moderate rotation
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.3),
    GaussianBlur(blur_limit=(1, 2), p=0.2),  # Low probability (20%) and minimal blur
    RandomCrop(width=450, height=450, p=0.5),  # Assuming your images are larger than this
], bbox_params=BboxParams(format='yolo', min_visibility=0.3))

# Function to read YOLO annotations
def read_yolo_annotations(txt_path, img_width, img_height):
    with open(txt_path, 'r') as f:
        lines = f.readlines()
    bboxes = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        bboxes.append([x_center, y_center, width, height, class_id])
    return bboxes

# Function to write YOLO annotations
def write_yolo_annotations(txt_path, bboxes):
    with open(txt_path, 'w') as f:
        for bbox in bboxes:
            line = f"{int(bbox[4])} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n"
            f.write(line)

# Process each image
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load image and corresponding annotation
        img_path = os.path.join(input_dir, filename)
        base_name = os.path.splitext(filename)[0]
        txt_path = os.path.join(input_dir, f"{base_name}.txt")
        
        image = cv2.imread(img_path)
        if image is None:
            continue
            
        height, width = image.shape[:2]
        bboxes = read_yolo_annotations(txt_path, width, height)
        
        # Generate 10 augmented versions
        for i in range(1, 11):
            # Apply augmentation
            augmented = aug(image=image, bboxes=bboxes)
            aug_image = augmented['image']
            aug_bboxes = augmented['bboxes']
            
            # Save augmented image
            aug_img_name = f"{base_name}_aug{i}.jpg"
            aug_img_path = os.path.join(output_dir, aug_img_name)
            cv2.imwrite(aug_img_path, aug_image)
            
            # Save augmented annotations
            aug_txt_name = f"{base_name}_aug{i}.txt"
            aug_txt_path = os.path.join(output_dir, aug_txt_name)
            write_yolo_annotations(aug_txt_path, aug_bboxes)

print("Data augmentation completed successfully!")

Data augmentation completed successfully!
